In [5]:
import geometry_definitions as gd
import openmc
import numpy as np
import pandas as pd

In [22]:
# From Linear Interpolation of Control Drum Data
critical_insertion_angle = 70.277
fuel_name = "graphite_fuel_435U_30C"
error_threshold = 100*10**-5
keff_trigger_dict = {'type': 'std_dev', 'threshold': error_threshold}
core_height = 89

# Grab Nuclear Data:
xs_loc = '/filespace/d/dokrueger/NE506/xs_data/endf_b-VII.0/endfb-viii.0-hdf5/cross_sections.xml'
openmc.config['cross_sections'] = xs_loc

# Generate the Model:
model = gd.get_model(core_height ,critical_insertion_angle)

model.settings.batches = 100
model.settings.particles = 10000
model.settings.temperature = {'default':2500}
model.settings.photon_transport = True
model.settings.keff_trigger = keff_trigger_dict
model.settings.trigger_batch_interval = 5
model.settings.trigger_max_batches = 100

# Create Initial Source:
# Measurements from Schnitzler 2007
inner_gap_inner_radius = 29.5275

lower_left = (-inner_gap_inner_radius, -inner_gap_inner_radius, -core_height/2)
upper_right = (inner_gap_inner_radius, inner_gap_inner_radius, core_height/2)
uniform_dist = openmc.stats.Box(lower_left, upper_right, only_fissionable = True)

# Set the source in settings
model.settings.source = openmc.source.Source(space=uniform_dist)

# ENDF/B-VIII.0 broad group library (56 energy groups):
energy_bins = [20000000, 6434000, 4304000, 3000000, 1850000, 1500000, 1200000, 861100, 750000,
                600000, 470000, 330000, 270000, 200000, 50000, 20000, 17000,
                3740, 2250, 191.5, 187.7, 117.5, 116, 105, 101.2,
                67.5, 65, 37.13, 36, 21.75, 21.2, 20.5, 7, 6.875,
                6.5, 6.25, 5, 1.13, 1.08, 1.01, 0.625, 0.45,
                0.375, 0.35, 0.325, 0.25, 0.2, 0.15, 0.1, 0.08,
                0.06, 0.05, 0.04, 0.0253, 0.01, 0.004]

energy_bins.reverse()

# Check which cell has the fuel, in case it ever changes for some reason...
for cell in model.geometry.get_all_material_cells().values():
    if cell.fill.name == fuel_name:
        fuel_cell_id = cell.id


# Set up Filters and triggers
fuel_cell_filter = openmc.DistribcellFilter(fuel_cell_id)
energy_filter = openmc.EnergyFilter(energy_bins)
flux_trigger = openmc.Trigger(trigger_type='rel_err', threshold=5.0)
heating_trigger = openmc.Trigger(trigger_type='rel_err', threshold=5.0)

# Set up tallies for flux and kappa-fission
heating_tally = openmc.Tally(name='Heating')
heating_tally.filters = [fuel_cell_filter]
heating_tally.scores = ['kappa-fission']
heating_tally.triggers = [heating_trigger]

flux_tally = openmc.Tally(name="Flux")
flux_tally.filters = [fuel_cell_filter, energy_filter]
flux_tally.scores = ['flux']
flux_tally.triggers = [flux_trigger]

model.tallies = [heating_tally, flux_tally]

model.export_to_model_xml()

statepoint = model.run(threads=8)

/usr/local/lib/python3.9/dist-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=14.
  warn(msg, IDWarning)
/usr/local/lib/python3.9/dist-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=11.
  warn(msg, IDWarning)
/usr/local/lib/python3.9/dist-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=10.
  warn(msg, IDWarning)
/usr/local/lib/python3.9/dist-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=7.
  warn(msg, IDWarning)
/usr/local/lib/python3.9/dist-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=15.
  warn(msg, IDWarning)
/usr/local/lib/python3.9/dist-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/usr/local/lib/python3.9/dist-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=8.
  wa

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

 Reading Si30 from
 /filespace/d/dokrueger/NE506/xs_data/endf_b-VII.0/endfb-viii.0-hdf5/neutron/Si30
 .h5
 Reading S33 from
 /filespace/d/dokrueger/NE506/xs_data/endf_b-VII.0/endfb-viii.0-hdf5/neutron/S33.
 h5
 Reading S from
 /filespace/d/dokrueger/NE506/xs_data/endf_b-VII.0/endfb-viii.0-hdf5/photon/S.h5
  Reading S34 from
 /filespace/d/dokrueger/NE506/xs_data/endf_b-VII.0/endfb-viii.0-hdf5/neutron/S34.
 h5
 Reading S36 from
 /filespace/d/dokrueger/NE506/xs_data/endf_b-VII.0/endfb-viii.0-hdf5/neutron/S36.
 h5
 Minimum neutron data temperature: 2500 K
 Maximum neutron data temperature: 2500 K
          This score will include contributions from all particles.
 Preparing distributed cell instances...
 Writing summary.h5 file...
 Maximum neutron transport energy: 20000000 eV for C13
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k       Entropy         Average k 
  =========   ========   ========   =======

In [18]:
print(keff_trigger_dict['threshold'])

0.01
